In [5]:
from sage.all import *

# ==========================================
# 1. 符号化整数类 (修复版)
# ==========================================

# 全局变量空间定义
DIM = 2048
F = GF(2)
# 这里不直接实例化 VectorSpace，为了避免每次调用开销，直接用 vector()

class SymInt:
    """
    模拟 64 位整数，使用 GF(2) 向量加法代替 XOR
    """
    def __init__(self, bits=None):
        if bits is None:
            # 默认为 0 (全零向量)
            self.bits = [vector(F, DIM) for _ in range(64)]
        else:
            self.bits = bits
            # 截断或填充至 64 位
            if len(self.bits) > 64:
                self.bits = self.bits[:64]
            elif len(self.bits) < 64:
                self.bits += [vector(F, DIM) for _ in range(64 - len(self.bits))]

    # !!! 关键修改：使用 __add__ 代替 __xor__ 避开 Sage 的 ^ 符号问题 !!!
    def __add__(self, other):
        # 对应位向量相加 (GF2 加法即异或)
        new_bits = [a + b for a, b in zip(self.bits, other.bits)]
        return SymInt(new_bits)

    def __lshift__(self, n):
        # 左移 n: 低位补零向量，高位丢弃
        if n == 0: return self
        if n >= 64: return SymInt()
        zero_vec = vector(F, DIM)
        new_bits = [zero_vec] * n + self.bits[:-n]
        return SymInt(new_bits)

    def __rshift__(self, n):
        # 右移 n: 高位补零向量，低位丢弃
        if n == 0: return self
        if n >= 64: return SymInt()
        zero_vec = vector(F, DIM)
        new_bits = self.bits[n:] + [zero_vec] * n
        return SymInt(new_bits)
    
    def rotate_left(self, n):
        # 循环左移: (x << n) | (x >> (64-n))
        n = n % 64
        if n == 0: return self
        # Python list index 0 is LSB.
        # Rotate Left n means: bit at index i comes from bit at index (i - n)
        new_bits = [self.bits[(i - n) % 64] for i in range(64)]
        return SymInt(new_bits)

# ==========================================
# 2. 模拟 RNG 逻辑 (使用 + 代替 ^)
# ==========================================

class SymbolicGiantLinearRNG:
    def __init__(self):
        self.state_size = 32
        self.state = []
        
        # 映射: Variable index k (0..2047) 对应 seed 的第 k 位
        # seed_int 是 Big Endian，但位操作是基于值的
        # state[0] 是 seed 的最高位部分
        
        for i in range(self.state_size):
            shift = 64 * (self.state_size - 1 - i)
            bits = []
            for bit_idx in range(64):
                # bit_idx 0 is LSB of this chunk
                # 对应 seed 的绝对位位置
                seed_bit_index = shift + bit_idx
                
                # 创建基向量
                v = vector(F, DIM)
                if seed_bit_index < DIM:
                    v[seed_bit_index] = 1
                bits.append(v)
            self.state.append(SymInt(bits))

    def next(self):
        s = self.state
        taps = [0, 1, 3, 7, 13, 22, 28, 31]
        
        new_val = SymInt() # 0
        
        for i in taps:
            val = s[i]
            # 原代码: mixed = val ^ ((val << 11) & mask) ^ (val >> 7)
            # 现代码: 使用 + 代替 ^
            mixed = val + (val << 11) + (val >> 7)
            
            rot = (i * 3) % 64
            # mixed = ((mixed << rot) | (mixed >> (64 - rot)))
            mixed = mixed.rotate_left(rot)
            
            new_val = new_val + mixed
 
        # new_val ^= (s[-1] >> 13) ^ ((s[-1] << 5) & mask)
        new_val = new_val + (s[-1] >> 13) + (s[-1] << 5)
        
        # 更新状态
        self.state = s[1:] + [new_val]
        
        # 计算输出
        out = SymInt()
        for i in range(self.state_size):
            if i % 2 == 0:
                out = out + self.state[i]
            else:
                val = self.state[i]
                # out ^= ((val >> 2) | (val << 62)) -> Rotate Left 62
                out = out + val.rotate_left(62)
                
        return out

# ==========================================
# 3. 主求解函数
# ==========================================

def solve():
    real_outputs = [
        11329270341625800450, 14683377949987450496, 11656037499566818711, 14613944493490807838, 
        370532313626579329, 5006729399082841610, 8072429272270319226, 3035866339305997883, 
        8753420467487863273, 15606411394407853524, 5092825474622599933, 6483262783952989294, 
        15380511644426948242, 13769333495965053018, 5620127072433438895, 6809804883045878003, 
        1965081297255415258, 2519823891124920624, 8990634037671460127, 3616252826436676639, 
        1455424466699459058, 2836976688807481485, 11291016575083277338, 1603466311071935653, 
        14629944881049387748, 3844587940332157570, 584252637567556589, 10739738025866331065, 
        11650614949586184265, 1828791347803497022, 9101164617572571488, 16034652114565169975, 
        13629596693592688618, 17837636002790364294, 10619900844581377650, 15079130325914713229, 
        5515526762186744782, 1211604266555550739, 11543408140362566331, 18425294270126030355, 
        2629175584127737886, 6074824578506719227, 6900475985494339491, 3263181255912585281, 
        12421969688110544830, 10785482337735433711, 10286647144557317983, 15284226677373655118, 
        9365502412429803694, 4248763523766770934, 13642948918986007294, 3512868807899248227, 
        14810275182048896102, 1674341743043240380, 28462467602860499, 1060872896572731679, 
        13208674648176077254, 14702937631401007104, 5386638277617718038, 8935128661284199759
    ]

    print("[*] Initializing Symbolic RNG...")
    rng = SymbolicGiantLinearRNG()
    
    matrix_rows = []
    target_vector = []
    
    # 取 40 个输出 (40 * 64 > 2048) 确保满秩
    print("[*] Collecting equations...")
    for k, real_val in enumerate(real_outputs[:40]):
        sym_val = rng.next()
        
        for bit_i in range(64):
            # 获取符号化系数 (向量)
            coeffs = sym_val.bits[bit_i]
            
            # 获取真实值比特
            target_bit = (real_val >> bit_i) & 1
            
            matrix_rows.append(coeffs)
            target_vector.append(target_bit)
            
    print(f"[*] Constructing Matrix ({len(matrix_rows)} x {DIM})...")
    
    M = Matrix(F, matrix_rows)
    b = vector(F, target_vector)
    
    print("[*] Solving linear system...")
    try:
        solution = M.solve_right(b)
    except ValueError as e:
        print("[-] Solver failed.")
        print(e)
        return

    print("[*] Reconstructing Flag...")
    
    seed_int = 0
    # solution 向量的索引 i 直接对应 seed_int 的第 i 位
    for i in range(DIM):
        if solution[i] == 1:
            seed_int |= (1 << i)
            
    try:
        flag_bytes = int(seed_int).to_bytes(256, 'big')
        flag = flag_bytes.replace(b'\x00', b'')
        print("\n" + "="*60)
        print("FLAG:", "0xfun{" + flag.decode(errors='ignore') + "}")
        print("="*60 + "\n")
    except Exception as e:
        print(f"[-] Decoding error: {e}")

if __name__ == '__main__':
    solve()

[*] Initializing Symbolic RNG...
[*] Collecting equations...
[*] Constructing Matrix (2560 x 2048)...
[*] Solving linear system...
[*] Reconstructing Flag...

FLAG: 0xfun{L1n34r_4lg3br4_W1th_Z3_1s_Aw3s0m3}

